# An overview of Gate Set Tomography

The `pygsti` package provides multiple levels of abstraction over the core Gate Set Tomography (GST) algorithms.  This tutorial will show you how to run Gate Set Tomography on some simulated (generated) data, hopefully giving you an overall sense of what it takes (and how easy it is!) to run GST.  For more details and options for running GST, see the [GST circuits tutorial](../objects/advanced/GSTCircuitConstruction.ipynb) and the [tutorial covering the different pyGSTi functions for running GST](GST-Drivers.ipynb).

To run GST, we need three inputs:
1. a "**target model**" which describes the desired, or ideal, operations we want our experimental hardware to perform.  In the example below, we use one of pyGSTi's "standard" models (see the [tutorial on standard modules](objects/advanced/StandardModules.ipynb)) - which acts on a single qubit with the following operations:
    - three gates: the identity, and $\pi/2$ rotations around the $x$- and $y$-axes.
    - a single state preparation in the $|0\rangle$ state.
    - a 2-outcome measurement with the label "0" associated with measuring $|0\rangle$ and "1" with measuring $|1\rangle$.
    
2. a list of circuits tailored to the target model; essentially a list of what experiments we need to run.  Using a standard model makes things especially straightforward here, since the building blocks, called *germ* and *fiducial* circuits, needed to make good GST circuits have already been computed (see the [tutorial on GST circuits](../objects/advanced/GSTCircuitConstruction.ipynb)).

3. data, in the form of experimental outcome counts, for each of the required sequences.  In this example we'll generate "fake" or "simulated" data from a depolarized version of our ideal model.  For more information about `DataSet` objects, see the [tutorial on DataSets](../objects/DataSet.ipynb).


In [1]:
#Make print statements compatible with Python 2 and 3
from __future__ import print_function

#Import the pygsti module (always do this) and the standard XYI model
import pygsti
from pygsti.construction import std1Q_XYI

# 1) get the target Model
target_model = std1Q_XYI.target_model()

# 2) get the building blocks needed to specify which operation sequences are needed
prep_fiducials, meas_fiducials = std1Q_XYI.prepStrs, std1Q_XYI.effectStrs
germs = std1Q_XYI.germs
maxLengths = [1,2,4,8,16,32] # roughly gives the length of the sequences used by GST

# 3) generate "fake" data from a depolarized version of target_model
mdl_datagen = target_model.depolarize(op_noise=0.01, spam_noise=0.001)
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(
    target_model, prep_fiducials, meas_fiducials, germs, maxLengths)
ds = pygsti.construction.generate_fake_data(mdl_datagen, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)

#Note: from listOfExperiments we can also create an empty dataset file
# which has columns of zeros where actual data should go. 
pygsti.io.write_empty_dataset("../tutorial_files/GettingStartedDataTemplate.txt", listOfExperiments,
                              "## Columns = 0 count, 1 count")
# After replacing the zeros with actual data, the data set can be 
# loaded back into pyGSTi using the line below and used in the rest
# of this tutorial. 
#ds = pygsti.io.load_dataset("tutorial_files/GettingStartedDataTemplate.txt")


Welcome to pygsti version 0.9.7!
There have been some major changes between this version and 0.9.6 - ones that break backward compatibility.  If you're trying to run an old script and nothing works, DON'T PANIC; we've tried to make the transition easy.  More often then not, you can just run `pyGSTi/scripts/upgrade2v0.9.7.py` on your old script or notebook files and you'll be up and running again.  For more information, see the pyGSTi FAQ.ipynb.

If this warning annoys you run:
  `export PYGSTI_BACKCOMPAT_WARNING=0` from the command line or
  `import os; os.environ['PYGSTI_BACKCOMPAT_WARNING'] = '0'` in a script or
     notebook *before* importing pygsti and the the madness will stop.


Now that we have all of the inputs, we can run GST in a standard way using the `do_stdpractice_gst` function.  For more information about this and related functions, see the [GST methods tutorial](GST-Drivers.ipynb).  This returns a `pygsti.report.Results` object (see the [Results tutorial](../objects/advanced/Results.ipynb)), from which we can generate a report giving us a summary of the analysis.   

In [2]:
#Run GST and create a report
results = pygsti.do_stdpractice_gst(ds, target_model, prep_fiducials, meas_fiducials, 
                                    germs, maxLengths, verbosity=3)

pygsti.report.create_standard_report(results, filename="../tutorial_files/gettingStartedReport",
                                    title="GST Overview Tutorial Example Report", verbosity=2)

-- Std Practice:  Iter 1 of 3  (TP) --: 
  --- Circuit Creation ---
     1702 sequences created
     Dataset has 1702 entries: 1702 utilized, 0 requested sequences were missing
  --- LGST ---
    Singular values of I_tilde (truncating to first 4 of 6) = 
    4.242865843377725
    1.3900765589279662
    1.3649838086380157
    1.3472944816189396
    0.04224890328547743
    0.030139799998574607
    
    Singular values of target I_tilde (truncating to first 4 of 6) = 
    4.242640687119285
    1.4142135623730963
    1.4142135623730958
    1.4142135623730954
    2.886142163912998e-16
    1.8096566263435588e-16
    
  --- Iterative MLGST: Iter 1 of 6  92 operation sequences ---: 
    --- Minimum Chi^2 GST ---
    Sum of Chi^2 = 82.6143 (92 data params - 31 model params = expected mean of 61; p-value = 0.0341509)
    Completed in 0.2s
    2*Delta(log(L)) = 83.985
    Iteration 1 took 0.2s
    
  --- Iterative MLGST: Iter 2 of 6  168 operation sequences ---: 
    --- Minimum Chi^2 GST ---
   

You can now open the file [../tutorial_files/gettingStartedReport/main.html](../tutorial_files/gettingStartedReport/main.html) in your browser (Firefox works best) to view the report.  **That's it!  You've just run GST!** 

In the cell above, `results` is a `Results` object, which is used to generate a HTML report.  For more information see the [Results object tutorial](../objects/advanced/Results.ipynb) and [report generation tutorial](../reporting/ReportGeneration.ipynb).

In [1]:
from IPython.display import IFrame

IFrame(src='../tutorial_files/gettingStartedReport/main.html', width=700, height=600)